In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.signal
from utils import audio, channels, synchronize, channel_estimate,ofdm, encode, decode, preamble,correction

cwd = os.getcwd()

In [ ]:
#baisc parameter definition
filename = 'output'            #rememeber to change everytime

chirp_duration = 1
record_duration = 50
fs = 48000

DFT_LENGTH = 4096
CP_LENGTH = 512
OFDM_LENGTH = 4608
symbol_per_frame = 128
low_freq = 1000
high_freq = 10000

encode_method = 'qpsk'
double_chirp = False
known_ofdm = True

#define chirp used in the signal
chirp_range = [1000,10000]
chirp = preamble.generate_chirp(chirp_duration, fs, low=chirp_range[0], high=chirp_range[1], silence_duration=0, double=double_chirp)

#define known ofdm symbol
known_ofdm = True
repeat_time = 4
known_ofdm_data = preamble.load_known_ofdm(CP_LENGTH,repeat_time)
spb = ofdm.subcarriers_per_block(fs,DFT_LENGTH,low_freq,high_freq)

In [ ]:
#load recorded signals
received_signal,_ = audio.wav_to_signal(filename,'audio_files')

In [ ]:
#synchronize: convolve and find
start,convolved = synchronize.chirp_synchronize(received_signal,chirp_range, fs,duration=1)
event_end = np.array(synchronize.impulse_detect(abs(convolved),fs,duration = 1,window_time=0.3,threshold=2))

#plot the convolved result
plt.plot(convolved)
for pos in event_end:
    plt.axvline(x = pos, color = 'r')
plt.show()

In [ ]:
#plot found chirp in received signal
event_start = event_end - fs*chirp_duration
# chirp_received = received_signal[event_start[0]:event_start[0]+chirp_duration*fs]   #taking only the first event for the time being

plt.plot(received_signal)
for pos in event_start:
    plt.axvline(x = pos, color = 'r')
plt.show()
print(len(event_start), " chirp events found at", event_start)

In [ ]:
event_start = event_start-5

In [ ]:
num_of_frames = len(event_start) // 2 - 1
events = np.array([ [event_start[2*i+1],event_start[2*i+2]] for i in range(num_of_frames) ])
pre_chirp_starts = events[:,0]
post_chirp_starts = events[:,1]
intervals = post_chirp_starts - pre_chirp_starts
frame_lengths = np.array([round((intervals[j]-chirp_duration*fs-2*known_ofdm_data.size)/OFDM_LENGTH) for j in range(num_of_frames)])
print("lengths of data frames: ", frame_lengths)
ori_lengths = frame_lengths * OFDM_LENGTH + 2*known_ofdm_data.size + chirp_duration*fs
sample_diffs = intervals - ori_lengths
print(str(num_of_frames)+" frames found, sample diff ",sample_diffs)

In [ ]:
#estimate with known ofdm
known_ofdm_start = (pre_chirp_starts + int(fs*(chirp_duration))) #need to change accordingly
known_ofdm_end = known_ofdm_start + known_ofdm_data.size

received_known = np.array([received_signal[known_ofdm_start[i]:known_ofdm_end[i]] for i in range(num_of_frames)]) #received ofdm part

#remove the first cyclic prefix manually then find discarded
received_known = received_known[:,CP_LENGTH:]

H_known_ofdm = [ofdm.known_ofdm_estimate_edited(received_known[i,:], 
known_ofdm_data[CP_LENGTH:CP_LENGTH+DFT_LENGTH], DFT_LENGTH, CP_LENGTH, low_freq, high_freq, fs) for i in range(num_of_frames)]
sigma = [decode.channel_noise(H_known_ofdm[i][1],received_known[i,:],known_ofdm_data,CP_LENGTH,DFT_LENGTH) for i in range(num_of_frames)]
print("noise estimation: ", sigma)

In [ ]:
#used for ofdm after a chirp only
ofdm_start = known_ofdm_end
ofdm_end = ofdm_start+frame_lengths*(OFDM_LENGTH)
deconvolved_list = []
for i in range(num_of_frames):
    received_ofdm = received_signal[ofdm_start[i]:ofdm_end[i]]
    
    #process received data
    fft = ofdm.ofdm_to_fourier(received_ofdm, DFT_LENGTH, CP_LENGTH)
    discarded = ofdm.subcarrier_extract(fft, DFT_LENGTH, fs, low_freq, high_freq)
    deconvolved = ofdm.deconvolve(discarded, H_known_ofdm[i][1], DFT_LENGTH, fs, low_freq, high_freq)
    deconvolved_list += [deconvolved]


In [ ]:
# phase correction
from IPython.display import clear_output
corrected_symbols = np.array([])
corrected_list  = []
for i in range(num_of_frames):

    #plot the phase difference between the two channel estimation done with known ofdm symbols
    phase_diff, H1, H2 = correction.phase_difference(received_signal, events[i,:],known_ofdm_data,CP_LENGTH,DFT_LENGTH,fs,low_freq,high_freq,repeat_time)
    phase_diff_unwrapped = correction.unwrap_phase(phase_diff)
    #find regression of the phase diff plot by looking at a selected linear range
    #iterate until satisfied with range for regression
    slope1, intercept1, r_value, p_value, std_err = scipy.stats.linregress(np.arange(spb), phase_diff_unwrapped)
    plt.plot(phase_diff_unwrapped)
    plt.plot(np.arange(spb)*slope1+intercept1)
    plt.title("Regression on unwrapped phase difference")
    plt.show()
    
    reply = None
    multi_regression = False
    while reply != 'y':
        reply = input("y: Proceed with unwrapped signal or type the range to perform regression").lower()
        if reply == 'y':
            slope = slope1
        else:
            regression_range_1 = np.array(reply.split()).astype(int)
        
            start1,end1 = regression_range_1
            if end1 > phase_diff.size:
                end1 = phase_diff.size
            plt.plot(phase_diff[start1:end1])
            section1 = phase_diff[start1:end1]
            x_1 = np.linspace(0,end1,num=(end1-start1))
            slope1, intercept1, r_value, p_value, std_err = scipy.stats.linregress(x_1, section1)
            plt.plot(x_1*slope1+intercept1)
            print(slope1,intercept1)
            plt.title("1st regression")
            plt.show()
            reply = input("confirm range? y/n").lower()
            if reply == 'y':
                multi_regression = True

    clear_output()
    if multi_regression:
        slope = correction.regression_correction(spb,slope1,intercept1,H1,H2,deconvolved_list[i],frame_lengths[i])


    full_sample_diff = sample_diffs[i] + slope * DFT_LENGTH / 2 / np.pi
    corrected = correction.phase_correction(deconvolved_list[i], full_sample_diff, DFT_LENGTH, CP_LENGTH, fs, low_freq, high_freq, chirp_duration)
    corrected_symbols = np.append(corrected_symbols, corrected)
    corrected_list.append(corrected)

In [ ]:
phase_plot_range = 3000
plt.scatter(np.real(corrected_symbols[:phase_plot_range]),np.imag(corrected_symbols[:phase_plot_range])) #plot the received phase distribution
print("Total symbols received: ", corrected_symbols.size)

In [ ]:
decoded = decode.qpsk_decode(corrected_symbols)

#check accuracy
new = False
if new:
    np.random.seed(8)
    encoded = np.random.randint(2, size=int(5*128*spb*2.75))
    np.savetxt('encoded', encoded, fmt='%i')
else:
    encoded = np.loadtxt('encoded', dtype=np.int32)

print(encoded.size, decoded.size)
print("source:\n", encoded[:30])
print("decoded:\n", decoded[:30])

In [ ]:
min_len = min(encoded.size, decoded.size)
equality = np.equal(encoded[:min_len], decoded[:min_len])
equality = equality[:]
print("correct rate:\n", np.sum(equality.astype(int))/equality.size)

In [ ]:
#error of each symbol
e_symbol = [0]*sum(frame_lengths)
for n in range(sum(frame_lengths)):
    a = equality[2*spb*n:2*spb*(n+1)]
    e_symbol[n] = np.round(np.sum(a.astype(int))/a.size,4)
plt.plot(e_symbol)
print(e_symbol[0])

In [ ]:
#ldpc decoding

result = [decode.ldpc_decode(corrected_list[i],sigma[i],spb)  for i in range(num_of_frames)]         #deals with one frame only
decoded_ldpc = np.concatenate(np.array([(result[i][0]<=0).astype(int) for i in range(num_of_frames) ]))
iteration = ([result[i][1] for i in range(num_of_frames) ])

#decoded info error rate
#info = np.loadtxt('info', dtype=np.int32)
info = 1

print(info[:30])
print(decoded_ldpc[:30])
    
equality_ldpc = info[:decoded_ldpc.size] == decoded_ldpc[:]
print("correct rate of decoded information:\n", np.sum(equality_ldpc.astype(int))/equality_ldpc.size) 

In [ ]:
print(info,decoded_ldpc.shape)

#print(iteration)

In [ ]:
from utils import file_preparation

file_preparation.file_decode(decoded_ldpc)